In [ ]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 6.2 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592285 sha256=4c59a0c41c27501dd0b171b9ae01ed2a0c28f5912f0310a0ecf4a7cbce78e04c
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [ ]:
import os
from aubio import source, notes
from mido import Message, MetaMessage, MidiFile, MidiTrack, second2tick, bpm2tempo

error_count = 0
files_not_converted = []

def convert_to_midi(input_file, output_file):
    global error_count

    try:
        downsample = 1
        samplerate = 44100 // downsample

        win_s = 512 // downsample # fft size
        hop_s = 256 // downsample # hop size

        s = source(input_file, samplerate, hop_s)
        samplerate = s.samplerate

        notes_o = notes("default", win_s, hop_s, samplerate)

        mid = MidiFile()
        track = MidiTrack()
        mid.tracks.append(track)

        ticks_per_beat = mid.ticks_per_beat
        bpm = 120

        tempo = bpm2tempo(bpm)
        track.append(MetaMessage('set_tempo', tempo=tempo))
        track.append(MetaMessage('time_signature', numerator=4, denominator=4))

        def frames2tick(frames, samplerate=samplerate):
            sec = frames / float(samplerate)
            return int(second2tick(sec, ticks_per_beat, tempo))

        last_time = 0
        total_frames = 0
        while True:
            samples, read = s()
            new_note = notes_o(samples)
            if (new_note[0] != 0):
                delta = frames2tick(total_frames) - last_time
                if new_note[2] > 0:
                    track.append(Message('note_off', note=int(new_note[2]),
                                         velocity=127, time=delta))
                track.append(Message('note_on',
                                     note=int(new_note[0]),
                                     velocity=int(new_note[1]),
                                     time=delta))
                last_time = frames2tick(total_frames)
            total_frames += read
            if read < hop_s: break

        mid.save(output_file)

    except (ValueError, RuntimeError) as e:
        print(f"Error processing {input_file}: {e}")
        error_count += 1
        files_not_converted.append(input_file)

input_dir = '/content/drive/MyDrive/genres_original'
output_dir = '/content/drive/MyDrive/output_midi'

for genre in os.listdir(input_dir):
    genre_path = os.path.join(input_dir, genre)
    if os.path.isdir(genre_path):
        output_genre_path = os.path.join(output_dir, genre)
        os.makedirs(output_genre_path, exist_ok=True)

        for file in os.listdir(genre_path):
            if file.endswith('.wav'):
                input_file = os.path.join(genre_path, file)
                output_file = os.path.join(output_genre_path, os.path.splitext(file)[0] + '.mid')
                convert_to_midi(input_file, output_file)

print("Number of files not converted due to errors:", len(files_not_converted))
# for file in files_not_converted:
#     print(file)


<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/blues/blues.00006.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/blues/blues.00003.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/blues/blues.00000.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/blues/blues.00007.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/blues/

Error processing /content/drive/MyDrive/genres_original/reggae/reggae.00040.wav: data byte must be in range 0..127


<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/reggae/reggae.00038.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/reggae/reggae.00037.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/reggae/reggae.00036.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/reggae/reggae.00035.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_origina

Error processing /content/drive/MyDrive/genres_original/hiphop/hiphop.00036.wav: data byte must be in range 0..127


<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/hiphop/hiphop.00037.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/hiphop/hiphop.00035.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/hiphop/hiphop.00028.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/hiphop/hiphop.00039.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_origina

Error processing /content/drive/MyDrive/genres_original/classical/classical.00080.wav: data byte must be in range 0..127


<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/classical/classical.00088.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/classical/classical.00087.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/classical/classical.00084.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/classical/classical.00079.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/driv

Error processing /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav: AUBIO ERROR: source_wavread: Failed opening /content/drive/MyDrive/genres_original/jazz/jazz.00054.wav (could not find RIFF header)


<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/jazz/jazz.00073.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/jazz/jazz.00072.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/jazz/jazz.00071.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/jazz/jazz.00070.wav from 22050 to 44100
  s = source(input_file, samplerate, hop_s)
<ipython-input-14-61b46632cc44>:18: UserWarning: AUBIO WARNING: source_avcodec: upsampling /content/drive/MyDrive/genres_original/jazz/jazz.0006

Number of files not converted due to errors: 4


In [ ]:
import os
import pandas as pd

# Base directory where the folders are located
base_dir = "/content/drive/MyDrive/output_midi"

# List of genres (folder names)
genres = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

# Initialize lists to hold file paths and genres
file_paths = []
file_genres = []

# Iterate over each genre and collect file paths
for genre in genres:
    genre_folder = os.path.join(base_dir, genre)
    for file in os.listdir(genre_folder):
        if file.endswith(".mid") or file.endswith(".midi"):
            file_paths.append(os.path.join(genre_folder, file))
            file_genres.append(genre)

# Create a DataFrame
df = pd.DataFrame({
    "Path": file_paths,
    "Genre": file_genres
})

# Save the DataFrame to a CSV file
csv_file = os.path.join(base_dir, "midi_files_genres.csv")
df.to_csv(csv_file, index=False)

print(f"CSV file created at {csv_file}")


CSV file created at /content/drive/MyDrive/output_midi/midi_files_genres.csv


In [ ]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import pretty_midi
import warnings
import os

In [ ]:
genre_df = pd.read_csv("/content/drive/MyDrive/output_midi/midi_files_genres.csv")
label_list = list(set(genre_df.Genre))
label_dict = {lbl: label_list.index(lbl) for lbl in label_list}

In [ ]:
def normalize_features(features):
    """
    This function normalizes the features to the range [-1, 1]

    @input features: The array of features.
    @type features: List of float

    @return: Normalized features.
    @rtype: List of float
    """
    tempo = (features[0] - 150) / 300
    num_sig_changes = (features[1] - 2) / 10
    resolution = (features[2] - 260) / 400
    time_sig_1 = (features[3] - 3) / 8
    time_sig_2 = (features[4] - 3) / 8
    return [tempo, resolution, time_sig_1, time_sig_2]


def get_features(path):
    """
    This function extracts the features from a midi file when given its path.

    @input path: The path to the midi file.
    @type path: String

    @return: The extracted features.
    @rtype: List of float
    """
    try:
        # Test for Corrupted Midi Files
        with warnings.catch_warnings():
            warnings.simplefilter("error")
            file = pretty_midi.PrettyMIDI(path)

            tempo = file.estimate_tempo()
            num_sig_changes = len(file.time_signature_changes)
            resolution = file.resolution
            ts_changes = file.time_signature_changes
            ts_1 = 4
            ts_2 = 4
            if len(ts_changes) > 0:
                ts_1 = ts_changes[0].numerator
                ts_2 = ts_changes[0].denominator
            return normalize_features([tempo, num_sig_changes, resolution, ts_1, ts_2])
    except:
        return None


def extract_midi_features(path_df):
    """
    This function takes in the path DataFrame, then for each midi file, it extracts certain
    features, maps the genre to a number and concatenates these to a large design matrix to return.

    @input path_df: A dataframe with paths to midi files, as well as their corresponding matched genre.
    @type path_df: pandas.DataFrame

    @return: A matrix of features along with label.
    @rtype: numpy.ndarray of float
    """
    all_features = []
    for index, row in path_df.iterrows():
        features = get_features(row.Path)
        genre = label_dict[row.Genre]
        if features is not None:
            features.append(genre)
            all_features.append(features)
    return np.array(all_features)


matched_midi_df = pd.read_csv("/content/drive/MyDrive/output_midi/midi_files_genres.csv")
labeled_features = extract_midi_features(matched_midi_df)
print(labeled_features)

[[ 0.08648339  0.55        0.125       0.125       5.        ]
 [ 0.19628286  0.55        0.125       0.125       5.        ]
 [ 0.16272189  0.55        0.125       0.125       5.        ]
 ...
 [-0.05296857  0.55        0.125       0.125       8.        ]
 [-0.08405546  0.55        0.125       0.125       8.        ]
 [-0.10925069  0.55        0.125       0.125       8.        ]]


In [ ]:
# Shuffle Entire Dataset to Make Random
labeled_features = np.random.permutation(labeled_features)

# Partition into 3 Sets
num = len(labeled_features)
num_training = int(num * 0.6)
num_validation = int(num * 0.8)
training_data = labeled_features[:num_training]
validation_data = labeled_features[num_training:num_validation]
test_data = labeled_features[num_validation:]

# Separate Features from Labels
num_cols = training_data.shape[1] - 1
training_features = training_data[:, :num_cols]
validation_features = validation_data[:, :num_cols]
test_features = test_data[:, :num_cols]

# Format Features for Multi-class Classification
num_classes = len(label_list)
training_labels = training_data[:, num_cols].astype(int)
validation_labels = validation_data[:, num_cols].astype(int)
test_labels = test_data[:, num_cols].astype(int)

# Function for One-Hot Encoding
def one_hot(labels):
    """
    This function encodes the labels using one-hot encoding.

    @input num_classes: The number of genres/classes.
    @type num_classes: int
    @input labels: The genre labels to encode.
    @type labels: numpy.ndarray of int

    @return: The one-hot encoding of the labels.
    @rtype: numpy.ndarray of int
    """
    return np.eye(num_classes)[labels].astype(int)

# Print to Check Dimentions and to Visualize
print(test_features[:10])
print(test_labels[:10])
print(one_hot(test_labels)[:10])

[[-0.06396669  0.55        0.125       0.125     ]
 [-0.24377224  0.55        0.125       0.125     ]
 [ 0.10520095  0.55        0.125       0.125     ]
 [ 0.09207048  0.55        0.125       0.125     ]
 [-0.09561921  0.55        0.125       0.125     ]
 [ 0.07595886  0.55        0.125       0.125     ]
 [ 0.14        0.55        0.125       0.125     ]
 [ 0.11302682  0.55        0.125       0.125     ]
 [-0.09843137  0.55        0.125       0.125     ]
 [ 0.14605678  0.55        0.125       0.125     ]]
[1 0 6 7 3 5 4 1 1 6]
[[0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]]


In [ ]:
def train_model(t_features, t_labels, v_features, v_labels):
    """
    This function trains a neural network using a couple different configurations.

    @input t_features: The training features.
    @type t_features: numpy.ndarray of float
    @input t_labels: The training labels.
    @type t_labels: numpy.ndarray of int
    @input v_features: The validation features.
    @type v_features: numpy.ndarray of float
    @input v_labels: The validation labels.
    @type v_labels: numpy.ndarray of int

    @return: The classifier that achieved the best validation accuracy.
    @rtype: sklearn.neural_network.multilayer_perceptron.MLPClassifier
    """
    # Neural Network and SVM Configurations
    clf_1 = MLPClassifier(solver='adam', alpha=1e-2, hidden_layer_sizes=(5,), random_state=1)
    clf_2 = MLPClassifier(solver='adam', alpha=1e-3, hidden_layer_sizes=(5, 5), random_state=1)
    clf_3 = MLPClassifier(solver='adam', alpha=1e-3, hidden_layer_sizes=(10, 10), random_state=1)
    clf_4 = MLPClassifier(solver='adam', alpha=1e-2, hidden_layer_sizes=(100, 100), random_state=1)
    clf_svm = RandomForestClassifier()

    # Keep Track of the Best Model
    best_clf = None
    best_accuracy = 0

    # Test the Accuracies of the Models and Get Best
    for clf in [clf_1, clf_2, clf_3, clf_4, clf_svm]:
        t_labels_hot = one_hot(t_labels)
        v_labels_hot = one_hot(v_labels)
        if (type(clf) == SVC):
            clf = clf.fit(t_features, t_labels)
        else:
            clf = clf.fit(t_features, t_labels_hot)
        predictions = clf.predict(v_features)
        count = 0
        for i in range(len(v_labels)):
            if (type(clf) != SVC):
                if np.array_equal(v_labels_hot[i], predictions[i]):
                    count += 1
            else:
                if v_labels[i] == predictions[i]:
                    count += 1
        accuracy = count / len(v_labels_hot)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_clf = clf

    print("Best Accuracy:", best_accuracy)
    return best_clf

classifier = train_model(training_features, training_labels, validation_features, validation_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Accuracy: 0.135678391959799
